In [1]:
item_id = '626ad59d42c6416ebb48f1aed495368f'

In [2]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import pandas as pd
import os

gis = GIS("home")

work in progress: https://chat.openai.com/g/g-dsjzmcL0j-python-gis-helper/c/9c0183fc-9466-4994-bddf-76b29935ce8f

In [3]:
# Function to get layer URL
def get_layer_url(item_id, layer_index):
    item = gis.content.get(item_id)
    if item and item.type == "Feature Service":
        layers = item.layers
        if len(layers) > layer_index:
            print(f"found {item_id}")
            return layers[layer_index].url
        else:
            print("Layer index out of range.")
            return None
    else:
        print("Item not found or is not a Feature Service.")
        return None

In [4]:
# Function to process and upload Excel files
def process_and_upload_excel(folder_name):
    user_items = gis.users.me.items(folder=folder_name)
    for item in user_items:
        if item.type == "Microsoft Excel":
            print(f"Processing File: {item.name}")
            file_path = item.download(save_path='/arcgis/home/')
            excel_file = pd.ExcelFile(file_path)
            first_sheet_name = excel_file.sheet_names[0]
            df = pd.read_excel(file_path, sheet_name=first_sheet_name)
            
            # Example of processing
            # Get data from Feature Services
            transmission_lines_url = get_layer_url(item_id, 0) # use injected parameter

            if transmission_lines_url:
                transmission_layer = FeatureLayer(transmission_lines_url)
                transmission_data = transmission_layer.query().sdf  # Get data as Spatial DataFrame
                # Perform data processing and merging with df
                print("Performing data processing and merging with df")
            
            # Format the current date and time (for ease of use during development)
            from datetime import datetime
            current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_filename = f'processed_{item.name.split(".")[0]}_{current_datetime}.xlsx'
            output_file_path = f'/arcgis/home/{output_filename}'
            
            # Save the processed DataFrame to a new Excel file
            df.to_excel(output_file_path, index=False)
            
            # Upload the new Excel file to the "scheduled_notebook_output" folder
            folder_name = "scheduled_notebook_output"
            output_folder = next((folder for folder in gis.users.me.folders if folder['title'].lower() == folder_name), None)
            if output_folder:
                # Check if "overwrite_me.xlsx" exists in the folder
                search_results = gis.content.search(query=f'title:"overwrite_me.xlsx" AND owner:{gis.users.me.username}', item_type='Microsoft Excel', outside_org=False)
                overwrite_item = None
                for item in search_results:
                    if item.title == 'overwrite_me.xlsx':
                        overwrite_item = item
                        break

                if overwrite_item:
                    # Overwrite existing item
                    overwrite_item.update(data=output_file_path)
                    print(f"Updated: {overwrite_item.title}")
                else:
                    # Upload as a new item if "overwrite_me.xlsx" does not exist
                    gis.content.add(item_properties={'title': output_filename, 'type': 'Microsoft Excel'},
                                    data=output_file_path,
                                    folder=output_folder['title'])
                    print(f"Uploaded: {output_filename}")
            else:
                print(f"{folder_name} folder not found.")

            # Clean up local files
            os.remove(file_path)
            os.remove(output_file_path)

The key to overwriting an existing item is to find the item by its name (or another unique identifier) and then use the `update()` method on the item object with the new file's path.

In [5]:
# Process files in the "enrichment" folder
enrichment_folder = next((folder for folder in gis.users.me.folders if folder['title'].lower() == "enrichment"), None)
if enrichment_folder:
    print(f"Processing files in 'enrichment' folder...")
    process_and_upload_excel(enrichment_folder['title'])
else:
    print("Enrichment folder not found.")

print("Process complete.")

Processing files in 'enrichment' folder...
Processing File: example.xlsx
found 626ad59d42c6416ebb48f1aed495368f
Performing data processing and merging with df
Updated: overwrite_me.xlsx
Processing File: example2.xlsx
found 626ad59d42c6416ebb48f1aed495368f
Performing data processing and merging with df
Updated: overwrite_me.xlsx
Process complete.
